In [1]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pyimagesearch.preprocessing import ImageToArrayPreprocessor
from pyimagesearch.preprocessing import AspectAwarePreprocessor
from pyimagesearch.datasets import SimpleDatasetLoader
from pyimagesearch.nn.conv import FCHeadNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from imutils import paths
import numpy as np
import os

In [2]:
args = {
    'dataset' : '/floyd/input/flowers_17',
    'model' : 'flowers17.model',
}

In [3]:
# construct the image data generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

In [4]:
# grab the list of image that we'll be describing, then extract the class label names from the image path
print("[info] loading images...")
imagePaths = list(paths.list_images(args['dataset']))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

[info] loading images...


In [5]:
# init the image preprocessor
aap = AspectAwarePreprocessor(224, 224)
iap = ImageToArrayPreprocessor()

# load the dataset from disk then scale the raw pixel intensifies to the range
# [0, 1]

sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

[INFO] processed 500/1360
[INFO] processed 1000/1360


In [6]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining for testing

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

lb = LabelBinarizer()
# convert the label from integer to vector
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [7]:
# load the VGG 16 network, ensuring the head FC layers sets are left off
baseModel = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))

# init the new head of the network, a set of FC layers followed by softmax classifier
headModel = FCHeadNet.build(baseModel, len(classNames), 256)

# place the head FC model on top of the base model -- this will become the actual model we will train
model = Model(inputs=baseModel.input, outputs=headModel)

58892288/58889256 [==============================] - 3s 0us/step


In [8]:
# loop over all layers in the base model and freeze them so they will not be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

In [9]:
# compile our model (this needs to be done after our setting our layers to be non-trainable)
print("[INFO] compiling model...")
opt = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# train the head of the network for a few epochs (all other layers are frozen) --
# this will allow the new FC layers to start to become init with actual learned values
# versus pure random

print(["[INFO] training head...."])
model.fit_generator(aug.flow(trainX, trainY, batch_size=32), validation_data=(testX, testY), epochs=25, steps_per_epoch=len(trainX) // 32, verbose=1)

[INFO] compiling model...
['[INFO] training head....']
Epoch 1/25
31/31 [==============================] - 105s 3s/step - loss: 7.4981 - accuracy: 0.1690 - val_loss: 1.9522 - val_accuracy: 0.3971
Epoch 2/25
31/31 [==============================] - 42s 1s/step - loss: 2.1722 - accuracy: 0.3421 - val_loss: 1.6336 - val_accuracy: 0.4618
Epoch 3/25
31/31 [==============================] - 41s 1s/step - loss: 1.8910 - accuracy: 0.4241 - val_loss: 1.1415 - val_accuracy: 0.6853
Epoch 4/25
31/31 [==============================] - 42s 1s/step - loss: 1.5327 - accuracy: 0.5282 - val_loss: 1.2398 - val_accuracy: 0.5912
Epoch 5/25
31/31 [==============================] - 42s 1s/step - loss: 1.4183 - accuracy: 0.5722 - val_loss: 0.7605 - val_accuracy: 0.7765
Epoch 6/25
31/31 [==============================] - 41s 1s/step - loss: 1.3339 - accuracy: 0.5786 - val_loss: 0.6608 - val_accuracy: 0.7882
Epoch 7/25
31/31 [==============================] - 42s 1s/step - loss: 1.1101 - accuracy: 0.6535 - val_

In [10]:
# evaluate network after init
print("[INFO] evaluating after init...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classNames))

[INFO] evaluating after init...
              precision    recall  f1-score   support

    bluebell       0.70      0.80      0.74        20
   buttercup       0.65      1.00      0.79        15
   coltsfoot       0.83      0.65      0.73        23
     cowslip       0.80      0.63      0.71        19
      crocus       0.86      0.90      0.88        21
    daffodil       0.93      0.81      0.87        16
       daisy       1.00      1.00      1.00        16
   dandelion       0.86      0.86      0.86        22
  fritillary       0.90      0.95      0.92        19
        iris       0.96      0.96      0.96        23
  lilyvalley       0.93      0.78      0.85        18
       pansy       0.93      0.96      0.95        27
    snowdrop       1.00      0.85      0.92        20
   sunflower       1.00      1.00      1.00        20
   tigerlily       0.90      1.00      0.95        18
       tulip       0.82      0.70      0.76        20
  windflower       0.85      1.00      0.92      

In [11]:
# now that the head FC layers have been trained lets unfreeze the final set of conv layers and make them trainable
for layer in baseModel.layers[15:]:
    layer.trainable = True

In [12]:
# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
print("[INFO] re-compiling model...")
opt = SGD(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])

# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
print("[INFO] fine-tuning model...")
model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
validation_data=(testX, testY), epochs=100,
steps_per_epoch=len(trainX) // 32, verbose=1)

[INFO] re-compiling model...
[INFO] fine-tuning model...
Epoch 1/100
31/31 [==============================] - 41s 1s/step - loss: 0.4007 - accuracy: 0.8623 - val_loss: 0.3907 - val_accuracy: 0.8824
Epoch 2/100
31/31 [==============================] - 41s 1s/step - loss: 0.3334 - accuracy: 0.8887 - val_loss: 0.3303 - val_accuracy: 0.9088
Epoch 3/100
31/31 [==============================] - 41s 1s/step - loss: 0.2839 - accuracy: 0.9079 - val_loss: 0.3332 - val_accuracy: 0.9059
Epoch 4/100
31/31 [==============================] - 41s 1s/step - loss: 0.2853 - accuracy: 0.9093 - val_loss: 0.2886 - val_accuracy: 0.9147
Epoch 5/100
31/31 [==============================] - 42s 1s/step - loss: 0.2523 - accuracy: 0.9126 - val_loss: 0.2868 - val_accuracy: 0.9206
Epoch 6/100
31/31 [==============================] - 41s 1s/step - loss: 0.2714 - accuracy: 0.9194 - val_loss: 0.2912 - val_accuracy: 0.9206
Epoch 7/100
31/31 [==============================] - 41s 1s/step - loss: 0.2212 - accuracy: 0.931

KeyboardInterrupt: 

In [13]:
# evaluate the network on the fine-tuned model
print("[INFO] evaluating after fine-tuning...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
predictions.argmax(axis=1), target_names=classNames))


# save the model to disk
print("[INFO] serializing model...")
model.save(args["model"])

[INFO] evaluating after fine-tuning...
              precision    recall  f1-score   support

    bluebell       1.00      0.90      0.95        20
   buttercup       0.93      0.93      0.93        15
   coltsfoot       0.85      0.74      0.79        23
     cowslip       1.00      0.74      0.85        19
      crocus       0.81      1.00      0.89        21
    daffodil       0.93      0.81      0.87        16
       daisy       1.00      1.00      1.00        16
   dandelion       0.90      0.86      0.88        22
  fritillary       1.00      1.00      1.00        19
        iris       0.96      0.96      0.96        23
  lilyvalley       0.94      0.89      0.91        18
       pansy       0.90      1.00      0.95        27
    snowdrop       1.00      0.95      0.97        20
   sunflower       1.00      1.00      1.00        20
   tigerlily       0.86      1.00      0.92        18
       tulip       0.69      0.90      0.78        20
  windflower       1.00      0.91      0.9